In [1]:
from core.database import engine

In [2]:
from splink.backends.postgres import PostgresAPI
from splink import SettingsCreator 
from splink import Linker, block_on
import splink.comparison_library as cl
from splink.blocking_analysis import count_comparisons_from_blocking_rule

In [80]:

comparison_name = {
    "output_column_name": "name",
    "comparison_levels": [
        {
            "sql_condition": "name_l IS NULL OR name_r IS NULL",
            "is_null_level": True,
        },
        {
            "sql_condition": "lower(name_l) = lower(name_r)",
            "label_for_charts": "Exact name (case-insensitive)",
        },
        {
            "sql_condition": "levenshtein(lower(name_l), lower(name_r)) <= 2",
            "label_for_charts": "Very close name",
        },
        {
            "sql_condition": "levenshtein(lower(name_l), lower(name_r)) <= 4",
            "label_for_charts": "Close name",
        },
        {"sql_condition": "ELSE"},
    ],
}


comparison_description = {
    "output_column_name": "description",
    "comparison_levels": [
        {
            "sql_condition": "description_l IS NULL OR description_r IS NULL",
            "is_null_level": True,
        },
        {
            "sql_condition": "similarity(description_l, description_r) >= 0.4",
            "label_for_charts": "High similarity",
            "tf_adjustment_weight": 0.6,
        },
        {
            "sql_condition": "similarity(description_l, description_r) >= 0.25",
            "label_for_charts": "Medium similarity",
            "tf_adjustment_weight": 0.3,
        },
        {"sql_condition": "ELSE"},
    ],
}




In [81]:
settings = SettingsCreator(
    link_type="dedupe_only",
    unique_id_column_name="id",
    blocking_rules_to_generate_predictions=[
        "LOWER(l.brand_name) = LOWER(r.brand_name)",
        "ABS(l.price - r.price) / NULLIF( ((l.price + r.price)/2), 0) <= 0.5",
    ],
    comparisons = [
        # name
        comparison_name,
        # description
        comparison_description,
    ]
)


In [82]:
dbapi = PostgresAPI(engine=engine)
linker = Linker(
    "raw_item",
    settings,
    db_api=dbapi,
)

In [83]:
pairwise_predictions = linker.inference.predict(
    threshold_match_weight=-5
)


Blocking time: 0.04 seconds
Predict time: 0.25 seconds

 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'name':
    m values not fully trained
Comparison: 'name':
    u values not fully trained
Comparison: 'description':
    m values not fully trained
Comparison: 'description':
    u values not fully trained
The 'probability_two_random_records_match' setting has been set to the default value (0.0001). 
If this is not the desired behaviour, either: 
 - assign a value for `probability_two_random_records_match` in your settings dictionary, or 
 - estimate with the `linker.training.estimate_probability_two_random_records_match` function.


In [84]:
pairwise_predictions.as_pandas_dataframe()

,match_weight,match_probability,id_l,id_r,name_l,name_r,gamma_name,description_l,description_r,gamma_description,brand_name_l,brand_name_r,price_l,price_r,match_key
0,-0.287568,0.450333,50,100,Mesa Escritório com Gaveta 120cm,Mesa Escritório c/ Gaveta 120cm,2,Mesa escritório com gaveta MDF 120x60cm branca,Com gaveta MDF branca 120cm largura 60cm profu...,2,Allma,Allma,520.00,515.00,0


In [56]:
df_pairs = pairwise_predictions.as_pandas_dataframe()

In [93]:
df_pairs[['name_l','name_r','description_l','description_r','match_probability']]

,name_l,name_r,description_l,description_r,match_probability
0,Cadeira Gamer RGB,Cadeira Gamer RGB Allma,Cadeira gamer com iluminação RGB e apoio para ...,Gamer iluminação RGB apoio pescoço lombar ajus...,0.999702
1,Mesa Escritório Compacta 100cm,Mesa Escritório Compacta 100cm Allma,Mesa escritório compacta MDF 100x50cm branca,Compacta MDF branca 100cm largura 50cm profund...,0.963258
2,Mesa Escritório Compacta 100cm,Mesa Escritório com Gaveta 120cm,Mesa escritório compacta MDF 100x50cm branca,Mesa escritório com gaveta MDF 120x60cm branca,0.092897
3,Mesa Escritório Formato L 140cm,Mesa Escritório Compacta 100cm Allma,Formato L MDF preta 140cm largura 70cm profund...,Compacta MDF branca 100cm largura 50cm profund...,0.092897
4,Mesa Escritório Formato L 140cm,Mesa Escritório c/ Gaveta 120cm,Formato L MDF preta 140cm largura 70cm profund...,Com gaveta MDF branca 120cm largura 60cm profu...,0.092897
...,...,...,...,...,...
100,Smartphone Xiaomi Redmi Note 12,Smartphone Redmi Note 12 128GB Xiaomi,"Smartphone 128GB 6GB RAM 6.43"" Android 12","128GB 6GB RAM 6.43"" AMOLED Android 12 câmera",0.999702
101,Fone de Ouvido Bluetooth Xiaomi,Fone Bluetooth In-Ear Xiaomi,Fone de ouvido bluetooth in-ear com case,Bluetooth in-ear com case carregamento carga r...,0.999702
102,Notebook Dell Inspiron 15 8GB,Notebook Lenovo IdeaPad 3,Notebook 15 polegadas Intel Core i5 8GB RAM 25...,"Notebook 15.6"" AMD Ryzen 5 8GB RAM 256GB SSD",0.092897
103,Caixa de Som JBL Go 3,Caixa de Som Bluetooth Xiaomi,Caixa de som bluetooth portátil à prova d'água,Caixa de som bluetooth portátil 16W,0.092897


In [94]:
df_pairs[df_pairs.match_probability >= 0.5][['name_l','name_r','description_l','description_r','match_probability']].sort_values(by="match_probability", ascending=False).to_csv("pairs.csv", index=False)

In [86]:
import networkx as nx

G = nx.Graph()

edges = df_pairs[df_pairs["match_probability"] >= 0.9][
    ["id_l", "id_r"]
].values

G.add_edges_from(edges)

clusters = list(nx.connected_components(G))

cluster_map = {
    node: cluster_id
    for cluster_id, nodes in enumerate(clusters)
    for node in nodes
}

In [ ]:
cluster_map

{np.int64(152): 0,
 np.int64(50): 0,
 np.int64(102): 0,
 np.int64(104): 1,
 np.int64(54): 1,
 np.int64(105): 2,
 np.int64(55): 2,
 np.int64(56): 3,
 np.int64(106): 3,
 np.int64(57): 4,
 np.int64(107): 4,
 np.int64(58): 5,
 np.int64(108): 5,
 np.int64(59): 6,
 np.int64(109): 6,
 np.int64(60): 7,
 np.int64(110): 7,
 np.int64(61): 8,
 np.int64(111): 8,
 np.int64(112): 9,
 np.int64(62): 9,
 np.int64(113): 10,
 np.int64(63): 10,
 np.int64(64): 11,
 np.int64(114): 11,
 np.int64(65): 12,
 np.int64(115): 12,
 np.int64(66): 13,
 np.int64(116): 13,
 np.int64(67): 14,
 np.int64(117): 14,
 np.int64(68): 15,
 np.int64(118): 15,
 np.int64(69): 16,
 np.int64(119): 16,
 np.int64(120): 17,
 np.int64(70): 17,
 np.int64(121): 18,
 np.int64(71): 18,
 np.int64(72): 19,
 np.int64(122): 19,
 np.int64(73): 20,
 np.int64(123): 20,
 np.int64(74): 21,
 np.int64(124): 21,
 np.int64(75): 22,
 np.int64(125): 22,
 np.int64(76): 23,
 np.int64(126): 23,
 np.int64(77): 24,
 np.int64(127): 24,
 np.int64(128): 25,
 np.in

In [92]:
for cluster_id, nodes in enumerate(clusters):
    print(f"\nCluster {cluster_id} ({len(nodes)} records)")
    display(df[df["node_id"].isin(nodes)][
        ["brand_name", "name", "price"]
    ])



Cluster 0 (3 records)


NameError: name 'df' is not defined

In [91]:
len(clusters)

50

In [87]:
cluster_map


{np.int64(152): 0,
 np.int64(50): 0,
 np.int64(102): 0,
 np.int64(104): 1,
 np.int64(54): 1,
 np.int64(105): 2,
 np.int64(55): 2,
 np.int64(56): 3,
 np.int64(106): 3,
 np.int64(57): 4,
 np.int64(107): 4,
 np.int64(58): 5,
 np.int64(108): 5,
 np.int64(59): 6,
 np.int64(109): 6,
 np.int64(60): 7,
 np.int64(110): 7,
 np.int64(61): 8,
 np.int64(111): 8,
 np.int64(112): 9,
 np.int64(62): 9,
 np.int64(113): 10,
 np.int64(63): 10,
 np.int64(64): 11,
 np.int64(114): 11,
 np.int64(65): 12,
 np.int64(115): 12,
 np.int64(66): 13,
 np.int64(116): 13,
 np.int64(67): 14,
 np.int64(117): 14,
 np.int64(68): 15,
 np.int64(118): 15,
 np.int64(69): 16,
 np.int64(119): 16,
 np.int64(120): 17,
 np.int64(70): 17,
 np.int64(121): 18,
 np.int64(71): 18,
 np.int64(72): 19,
 np.int64(122): 19,
 np.int64(73): 20,
 np.int64(123): 20,
 np.int64(74): 21,
 np.int64(124): 21,
 np.int64(75): 22,
 np.int64(125): 22,
 np.int64(76): 23,
 np.int64(126): 23,
 np.int64(77): 24,
 np.int64(127): 24,
 np.int64(128): 25,
 np.in

In [40]:
br_price = "ABS(l.price - r.price) / NULLIF(GREATEST(l.price, r.price), 0) <= 0.8"
br_brand = "LOWER(l.brand_name) = LOWER(r.brand_name)" # can this be more flexible like to lower case brand names (levenshtein(l.brand_name, r.brand_name) < 3)

In [41]:
count_comparisons_from_blocking_rule(
        table_or_tables='raw_item',
        unique_id_column_name="id",
        blocking_rule=br_brand,
        link_type="dedupe_only",
        db_api=dbapi,
    )

{'number_of_comparisons_generated_pre_filter_conditions': 2025,
 'number_of_comparisons_to_be_scored_post_filter_conditions': 936,
 'filter_conditions_identified': '',
 'equi_join_conditions_identified': 'LOWER(l.brand_name) = LOWER(r.brand_name)',
 'link_type_join_condition': 'where l."id" < r."id"'}

In [42]:
count_comparisons_from_blocking_rule(
        table_or_tables='raw_item',
        unique_id_column_name="id",
        blocking_rule=br_price,
        link_type="dedupe_only",
        db_api=dbapi,
    )

{'number_of_comparisons_generated_pre_filter_conditions': 23409,
 'number_of_comparisons_to_be_scored_post_filter_conditions': 6232,
 'filter_conditions_identified': 'ABS(l.price - r.price) / NULLIF(GREATEST(l.price, r.price), 0) <= 0.8',
 'equi_join_conditions_identified': '',
 'link_type_join_condition': 'where l."id" < r."id"'}

In [ ]:
cl.LevenshteinAtThresholds(col_name='brand', thresholds=[1,2,3])
cl.CosineSimilarityAtThresholds(col_name='description', thresholds=[0.4, 0.6, 0.8])
cl.LevenshteinAtThresholds(col_name='brand', thresholds=[1,2,3])

In [ ]:
cl.JaroWinklerAtThresholds

In [ ]:
import splink.comparison_library as cl
from splink import DuckDBAPI, Linker, SettingsCreator, block_on, splink_datasets

db_api = DuckDBAPI()

df = splink_datasets.fake_1000

settings = SettingsCreator(
    link_type="dedupe_only",
    comparisons=[
        cl.NameComparison("first_name"),
        cl.JaroAtThresholds("surname"),
        cl.DateOfBirthComparison(
            "dob",
            input_is_string=True,
        ),
        cl.ExactMatch("city").configure(term_frequency_adjustments=True),
        cl.EmailComparison("email"),
    ],
    blocking_rules_to_generate_predictions=[
        block_on("first_name", "dob"),
        block_on("surname"),
    ]
)

linker = Linker(df, settings, db_api)

linker.training.estimate_probability_two_random_records_match(
    [block_on("first_name", "surname")],
    recall=0.7,
)

linker.training.estimate_u_using_random_sampling(max_pairs=1e6)

linker.training.estimate_parameters_using_expectation_maximisation(
    block_on("first_name", "surname")
)

linker.training.estimate_parameters_using_expectation_maximisation(block_on("email"))

pairwise_predictions = linker.inference.predict(threshold_match_weight=-5)

clusters = linker.clustering.cluster_pairwise_predictions_at_threshold(
    pairwise_predictions, 0.95
)

df_clusters = clusters.as_pandas_dataframe(limit=5)